In [1]:
from pathlib import Path
import requests, zipfile, io

url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/w767nnk5wh-1.zip"
base_folder = Path.cwd() / "Data"
zip_path = base_folder / "intan_data.zip"
extract_to = base_folder / "intan_data"


print("Downloading File...")
response = requests.get(url, stream=True)
response.raise_for_status()

with open(zip_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Download complete: {zip_path}")

# 3. Unzip to a folder
print("Extracting files...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")

Download complete: /home/ubuntu/codespace/sorter-script/Data/intan_data.zip
Extracting files...
Files extracted to: /home/ubuntu/codespace/sorter-script/Data/intan_data


In [23]:
from spikeinterface.sorters import run_sorter
import spikeinterface.full as si
import probeinterface as pi
from pathlib import Path

base_folder = Path.cwd() / "Data"
intan_file = base_folder / "intan_data/Intan RHD 2000 file of electrophysiological recordings/Intan RHD file1.rhd"

# Load Recording
rec = si.read_intan(intan_file, stream_id = "0")
rec

IntanRecordingExtractor: 16 channels - 20.0kHz - 1 segments - 24,000,480 samples 
                         1,200.02s (20.00 minutes) - uint16 dtype - 732.44 MiB
  file_path: /home/ubuntu/codespace/sorter-script/Data/intan_data/Intan RHD 2000 file of electrophysiological recordings/Intan RHD file1.rhd

In [ ]:
# Create custom probe geometry

probe = pi.Probe(ndim=2)
positions = []

for i in range(16):
    positions.append([0, i * 50])
probe.set_contacts(positions = positions, shapes = "circle", shape_params = {'radius':5})

probe.set_device_channel_indices(range(16))
probe.set_contact_ids([f"ch{i}" for i in range(16)])

probe_path = "neuronexus-A16x1_2mm_50_177_A16.json"
pi.write_probeinterface(probe_path, probe)


In [ ]:
# Attach Probe to Recording
rec = rec.set_probe(probe)

n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")

In [32]:
# Run Kilosort
sorting_KS4 = run_sorter(
    sorter_name="kilosort4",
    recording=rec,
    folder="Data/Sorter_output",
    verbose=True,
)

write_binary_recording (no parallelization): 100%|██████████| 1201/1201 [00:08<00:00, 144.88it/s]
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 24000480
kilosort.run_kilosort: N seconds: 1200.024
kilosort.run_kilosort: N batches: 401
kilosort.run_kilosort: Preprocessing filters computed in 0.42s; total 0.42s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     5.30 %
kilosort.run_kilosort: Mem used:     12.50 %     |       7.75 GB
kilosort.run_kilosort: Mem avail:    54.35 / 62.10 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   15.44 %     |      2.25   /    14.56 GB

kilosort4 run time 46.21s


In [41]:
import spikeinterface.full as si

base_folder = Path.cwd() / "Data"
intan_file = base_folder / "intan_data/Intan RHD 2000 file of electrophysiological recordings/Intan RHD file1.rhd"

# Load Recording
recording = si.read_intan(intan_file, stream_id = "0")
recording = recording.set_probe(probe, in_place=False)
recording = si.unsigned_to_signed(recording)
recording_filtered = si.bandpass_filter(recording)

job_kwargs = dict(n_jobs=-1, progress_bar=True, chunk_duration="1s")

sorting_analyzer = si.create_sorting_analyzer(sorting_KS4, recording_filtered, overwrite = True,
format="binary_folder", folder="Data/intan_analyzer",
**job_kwargs )
sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
sorting_analyzer.compute("waveforms", **job_kwargs)
sorting_analyzer.compute("templates", **job_kwargs)
sorting_analyzer.compute("noise_levels")
sorting_analyzer.compute("unit_locations", method = "monopolar_triangulation")
sorting_analyzer.compute("isi_histograms")
sorting_analyzer.compute("correlograms", window_ms=100, bin_ms=5)
sorting_analyzer.compute("principal_components", n_components=3, mode="by_channel_global", whiten=True, **job_kwargs)
sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
sorting_analyzer.compute("template_similarity")
sorting_analyzer.compute("spike_amplitudes", **job_kwargs)

spike_amplitudes (workers: 16 processes): 100%|██████████| 1201/1201 [00:01<00:00, 648.90it/s]


In [ ]:
import spikeinterface.full as si
from spikeinterface_gui import run_mainwindow

sorting_analyzer = si.load_sorting_analyzer(folder="Data/intan_analyzer")

run_mainwindow(sorting_analyzer, mode="web", curation=True)

/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/utils_panel.py:12: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/unitlist.py:436: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/mergeview.py:282: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/curationview.py:216: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("tabulator")


/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface_gui/backend_panel.py:253: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension("gridstack")


Found available port: 56439
Launching server at http://localhost:56439


ERROR:tornado.application:Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x78cc081efd10>>, <Task finished name='Task-31649' coro=<ServerSession.with_document_locked() done, defined at /home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/bokeh/server/session.py:77> exception=RuntimeError("Models must be owned by only a single document, Range1d(id='33ba933f-b522-4932-91cf-a18b40c98108', ...) is already in a doc")>)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/tornado/ioloop.py", line 782, in _discard_future_result
    future.result()
  File "/home/ubuntu/miniconda3/envs/sorter/lib/python3.11/site-packages/bokeh/server/session.py", line 94, in _needs_document_lock_w